In [1]:
#!/usr/bin/env python
import os
import math
import random
import numpy as np
from threading import Lock


import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import rospy
import rosservice

from nav_msgs.msg import Odometry
from std_msgs.msg import String
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist, Point

from std_srvs.srv import Empty
from pcimr_simulation.srv import InitPos
from gazebo_msgs.srv import GetModelState, GetModelStateRequest, SetModelState, SetModelStateRequest

SPEED_X = 0.7
SPEED_Y = 0.5
ANGULAR_Z = 0.4


class ControllerNode:

    def __init__(self):
        # Initialize member variables
        self.sim_lock = Lock()

        self.sub_twist = rospy.Subscriber('/input/cmd_vel', Twist, self.get_and_filter_twist)
        self.sub_scan = rospy.Subscriber('/scan', LaserScan, self.get_scan)
            
        self.pub_twist = rospy.Publisher('/cmd_vel', Twist, queue_size=10)

        self.scan = LaserScan()
        self.robot_twist = Twist()
        self.cmd_vel = Twist()
        
        #the service that lets you get the state of a model withing the gazebo environment (including position)
        rospy.wait_for_service ('/gazebo/get_model_state')
        self.get_model_srv = rospy.ServiceProxy('/gazebo/get_model_state', GetModelState)

        #state requests withg the names of the respective model
        self.robot_model = GetModelStateRequest()
        self.robot_model.model_name='rto-1'

        self.goal_model = GetModelStateRequest()
        self.goal_model.model_name='goal'


    def get_and_filter_twist(self, msg):
        self.robot_twist = msg
    
    def get_scan(self, msg):
        # TODO: discritize sensor data
        self.scan = msg
        
    def get_env(self):
        rospy.wait_for_message("scan", LaserScan)
        x = list(self.scan.ranges[0::35])+ [self.check_goal_distance()]
        return torch.FloatTensor(x)
    
    #get the distance between the robot "rto-1" and "goal" object
    def check_goal_distance(self):
        result = self.get_model_srv(self.robot_model)
        robot_pos = result.pose.position

        result = self.get_model_srv(self.goal_model)
        goal_pos = result.pose.position

        diff_x = robot_pos.x - goal_pos.x
        diff_y = robot_pos.y - goal_pos.y

        return math.sqrt(diff_x*diff_x + diff_y*diff_y)
    
    def get_reward(self):
        rospy.wait_for_message("scan", LaserScan)
        if min(self.scan.ranges) < 0.15:
            return -1
        elif min(self.scan.ranges) > 1.0:
            return -0.01 + 0.01 / (self.check_goal_distance() - 4.9)
        elif self.check_goal_distance() < 5.0:
            return +1
        else:
            return -0.05 +  0.04 / (self.check_goal_distance() - 4.9)
        
    def use_action(self, nr):
        if nr == 0:
            self.cmd_vel.linear.x = SPEED_X
            self.cmd_vel.linear.y = 0.0
            self.cmd_vel.angular.z = 0.0
        if nr == 1:
            self.cmd_vel.linear.x = SPEED_X
            self.cmd_vel.linear.y = 0.0
            self.cmd_vel.angular.z = ANGULAR_Z
        if nr == 2:
            self.cmd_vel.linear.x = SPEED_X
            self.cmd_vel.linear.y = 0.0
            self.cmd_vel.angular.z = -ANGULAR_Z
        self.pub_twist.publish(self.cmd_vel)
            



In [2]:



# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/deniz/praktikum_final/env/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:
class DQN(nn.Module):

    def __init__(self, outputs):
        super(DQN, self).__init__()
        # TODO: Change this numbers for input
        self.fc1 = nn.Linear(8, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 200)
        self.fc4 = nn.Linear(200, 3)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        #x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))

        x = self.fc4(x)
        
        return x

In [5]:
# TODO : Check this
BATCH_SIZE = 32
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

n_actions = 3


policy_checkpoint_path = None# "/home/deniz/praktikum_final/catkin_ws/src/robotino_rl/models/policy_net_10.pth"
target_checkpoint_path = None# "/home/deniz/praktikum_final/catkin_ws/src/robotino_rl/models/target_net_10.pth"

policy_net = DQN(n_actions).to(device)
target_net = DQN(n_actions).to(device)
# target_net.load_state_dict(policy_net.state_dict())


if policy_checkpoint_path != None:
    print("loading network")
    #load policy net
    policy_checkpoint = torch.load(policy_checkpoint_path)
    policy_net.load_state_dict(policy_checkpoint['model_state_dict'])
    optimizer.load_state_dict(policy_checkpoint['optimizer_state_dict'])
    
    #load target net
    target_checkpoint = torch.load(target_checkpoint_path)
    target_net.load_state_dict(target_checkpoint['model_state_dict'])
else:
    target_net.load_state_dict(policy_net.state_dict())

target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)

steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            t = policy_net(state)
            return torch.tensor([[t.max(0)[1]]])
    else:
        t = torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)
        return t


episode_durations = []


In [6]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])

    
    
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_batch=torch.reshape(state_batch, (32,8))
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    non_final_next_states=torch.reshape(non_final_next_states, (32,8))
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.data.clamp_(-1, 1)
    optimizer.step()

In [ ]:
num_episodes = 1000
print("Setp")
reset_simulation = rospy.ServiceProxy('/gazebo/reset_simulation', Empty)
reset_simulation()
rospy.init_node('controller_node')

simple_sim_node = ControllerNode()
checkpoint_root = "/home/deniz/praktikum_final/catkin_ws/src/robotino_rl/models"
for i_episode in range(num_episodes):
    if i_episode%100 == 0 and i_episode != 0:
        print("saving model after episode " + str(i_episode))
        
        #save policy net
        torch.save({
            'model_state_dict': policy_net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, checkpoint_root + "/policy_net_" + str(i_episode) + ".pth")
        
        #save target net
        torch.save({
            'model_state_dict': policy_net.state_dict()
            }, checkpoint_root + "/target_net_" + str(i_episode) + ".pth")
    #   Initialize the environment and state
    last_screen = simple_sim_node.get_env()
    current_screen = simple_sim_node.get_env()
    state =current_screen - last_screen
    for t in count():
        # Select and perform an action
        action = select_action(state)
        # Get values from action
        simple_sim_node.use_action(action.item())
        reward = simple_sim_node.get_reward()
        reward = torch.tensor([reward], device=device)
        
        last_screen = current_screen
        current_screen = simple_sim_node.get_env()
        
        next_state = current_screen - last_screen

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state
        
        # Perform one step of the optimization (on the target network)
        optimize_model()
        if reward == -1:
            episode_durations.append(t + 1)
            reset_simulation()
            break
        if reward == +1:
            print(f"reached goal at {i_episode}th episode {t}")
            episode_durations.append(t + 1)
            reset_simulation()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')

Setp
reached goal at 70th episode 246
reached goal at 83th episode 206
reached goal at 86th episode 269
saving model after episode 100
reached goal at 147th episode 233
